In [1]:
%%capture
!pip install titiler

In [5]:
import pandas as pd
import os
import s3fs
from IPython.display import display, HTML

import test_helpers

In [8]:
s3fsfs = s3fs.S3FileSystem(
    key=os.environ['AWS_ACCESS_KEY_ID'],
    secret=os.environ['AWS_SECRET_ACCESS_KEY'],
    default_fill_cache=False
)
test_files = test_helpers.list_test_files(s3fsfs = s3fsfs, file_pattern="time6*_Tair.nc")
test_files

['nasa-waterinsight/NLDAS3/forcing/rechunked_test/single_variable/time6_lat1000_lon1800_Tair.nc',
 'nasa-waterinsight/NLDAS3/forcing/rechunked_test/single_variable/time6_lat100_lon180_Tair.nc',
 'nasa-waterinsight/NLDAS3/forcing/rechunked_test/single_variable/time6_lat2000_lon3600_Tair.nc',
 'nasa-waterinsight/NLDAS3/forcing/rechunked_test/single_variable/time6_lat200_lon360_Tair.nc']

In [4]:
block_size_options = [
    1 * 10**6, # 1MB
    5 * 10**6, # 5MB
    10 * 10**6, # 10MB
    5 * 10**7, # 50MB
    10 * 10**7, # 100MB
]

# 0.01 degree x 0.01 degree timeseries

96 bytes of data

In [5]:
# these 2 need to match, precision equaling the spatial size's decimal points
spatial_size = 0.01
precision = 2
s3_times = {}
open_and_load_times = []
lat_range, lon_range = test_helpers.select_random_spatial_range(spatial_size, precision)

In [6]:
for block_size in block_size_options:
    for test_file in test_files:
        s3fs_open_kwargs = {
            'default_fill_cache': False,
            'default_block_size': block_size,
        }
        s3fsfs = s3fs.S3FileSystem(**s3fs_open_kwargs)
        test_info, s3_request_info = test_helpers.load_series(test_file, s3fsfs, lat_range, lon_range)
        block_size_mb = round(block_size/1024/1024, 3)
        test_info['s3fs_default_blocksize (mb)'] = block_size_mb
        open_and_load_times.append(test_info)
        s3_times[str(block_size_mb)] = s3_request_info

starting test for time6_lat1000_lon1800_Tair.nc
starting test for time6_lat100_lon180_Tair.nc
starting test for time6_lat2000_lon3600_Tair.nc
starting test for time6_lat200_lon360_Tair.nc
starting test for time6_lat1000_lon1800_Tair.nc
starting test for time6_lat100_lon180_Tair.nc
starting test for time6_lat2000_lon3600_Tair.nc
starting test for time6_lat200_lon360_Tair.nc
starting test for time6_lat1000_lon1800_Tair.nc
starting test for time6_lat100_lon180_Tair.nc
starting test for time6_lat2000_lon3600_Tair.nc
starting test for time6_lat200_lon360_Tair.nc
starting test for time6_lat1000_lon1800_Tair.nc
starting test for time6_lat100_lon180_Tair.nc
starting test for time6_lat2000_lon3600_Tair.nc
starting test for time6_lat200_lon360_Tair.nc
starting test for time6_lat1000_lon1800_Tair.nc
starting test for time6_lat100_lon180_Tair.nc
starting test for time6_lat2000_lon3600_Tair.nc
starting test for time6_lat200_lon360_Tair.nc


In [7]:
pd.DataFrame(open_and_load_times).sort_values(by='total_time')

filename                            chunk_shape  \
1     time6_lat100_lon180_Tair.nc    {'time': 6, 'lat': 100, 'lon': 180}   
3     time6_lat200_lon360_Tair.nc    {'time': 6, 'lat': 200, 'lon': 360}   
7     time6_lat200_lon360_Tair.nc    {'time': 6, 'lat': 200, 'lon': 360}   
5     time6_lat100_lon180_Tair.nc    {'time': 6, 'lat': 100, 'lon': 180}   
11    time6_lat200_lon360_Tair.nc    {'time': 6, 'lat': 200, 'lon': 360}   
9     time6_lat100_lon180_Tair.nc    {'time': 6, 'lat': 100, 'lon': 180}   
4   time6_lat1000_lon1800_Tair.nc  {'time': 6, 'lat': 1000, 'lon': 1800}   
8   time6_lat1000_lon1800_Tair.nc  {'time': 6, 'lat': 1000, 'lon': 1800}   
0   time6_lat1000_lon1800_Tair.nc  {'time': 6, 'lat': 1000, 'lon': 1800}   
6   time6_lat2000_lon3600_Tair.nc  {'time': 6, 'lat': 2000, 'lon': 3600}   
10  time6_lat2000_lon3600_Tair.nc  {'time': 6, 'lat': 2000, 'lon': 3600}   
2   time6_lat2000_lon3600_Tair.nc  {'time': 6, 'lat': 2000, 'lon': 3600}   
12  time6_lat1000_lon1800_Tair.nc  {'time': 6, 'lat': 1000, 'lon': 1800}   
15    time6_lat200_lon360_Tair.nc    {'time': 6, 'lat': 200, 'lon': 360}   
13    time6_lat100_lon180_Tair.nc    {'time': 6, 'lat': 100, 'lon': 180}   
16  time6_lat1000_lon1800_Tair.nc  {'time': 6, 'lat': 1000, 'lon': 1800}   
14  time6_lat2000_lon3600_Tair.nc  {'time': 6, 'lat': 2000, 'lon': 3600}   
18  time6_lat2000_lon3600_Tair.nc  {'time': 6, 'lat': 2000, 'lon': 3600}   
19    time6_lat200_lon360_Tair.nc    {'time': 6, 'lat': 200, 'lon': 360}   
17    time6_lat100_lon180_Tair.nc    {'time': 6, 'lat': 100, 'lon': 180}   

    chunk_size (mb)  open (seconds)       load  total_time  \
1          0.411987        0.126578   0.876857    1.003435   
3          1.647949        0.160635   0.969582    1.130217   
7          1.647949        0.203011   1.424826    1.627837   
5          0.411987        0.428542   1.430497    1.859039   
11         1.647949        0.168874   1.944920    2.113794   
9          0.411987        0.405387   1.835830    2.241217   
4         41.198730        0.401586   2.651872    3.053458   
8         41.198730        0.759598   2.542576    3.302174   
0         41.198730        2.652933   1.974212    4.627145   
6        164.794922        0.238345   6.147694    6.386039   
10       164.794922        0.196834   6.512931    6.709765   
2        164.794922        0.119699   7.729856    7.849555   
12        41.198730        1.195527   6.808832    8.004359   
15         1.647949        1.242421   7.761763    9.004184   
13         0.411987        0.901432   8.109964    9.011396   
16        41.198730        2.234302   8.637646   10.871948   
14       164.794922        1.531601  10.536037   12.067638   
18       164.794922        1.978936  11.429094   13.408030   
19         1.647949        1.926575  13.275389   15.201964   
17         0.411987        1.820072  13.419377   15.239449   

    s3fs_default_blocksize (mb)  
1                         0.954  
3                         0.954  
7                         4.768  
5                         4.768  
11                        9.537  
9                         9.537  
4                         4.768  
8                         9.537  
0                         0.954  
6                         4.768  
10                        9.537  
2                         0.954  
12                       47.684  
15                       47.684  
13                       47.684  
16                       95.367  
14                       47.684  
18                       95.367  
19                       95.367  
17                       95.367

# 1 degree x 1 degree timeseries

960 kilobytes of data

In [8]:
# these 2 need to match, precision equaling the spatial size's decimal points
spatial_size = 1
precision = 0
s3_times = {}
open_and_load_times = []
lat_range, lon_range = test_helpers.select_random_spatial_range(spatial_size, precision)

In [9]:
for block_size in block_size_options:
    for test_file in test_files:
        s3fs_open_kwargs = {
            'default_fill_cache': False,
            'default_block_size': block_size,
        }
        s3fsfs = s3fs.S3FileSystem(**s3fs_open_kwargs)
        test_info, s3_request_info = test_helpers.load_series(test_file, s3fsfs, lat_range, lon_range)
        block_size_mb = round(block_size/1024/1024, 3)
        test_info['s3fs_default_blocksize (mb)'] = block_size_mb
        open_and_load_times.append(test_info)
        s3_times[str(block_size_mb)] = s3_request_info

starting test for time6_lat1000_lon1800_Tair.nc
starting test for time6_lat100_lon180_Tair.nc
starting test for time6_lat2000_lon3600_Tair.nc
starting test for time6_lat200_lon360_Tair.nc
starting test for time6_lat1000_lon1800_Tair.nc
starting test for time6_lat100_lon180_Tair.nc
starting test for time6_lat2000_lon3600_Tair.nc
starting test for time6_lat200_lon360_Tair.nc
starting test for time6_lat1000_lon1800_Tair.nc
starting test for time6_lat100_lon180_Tair.nc
starting test for time6_lat2000_lon3600_Tair.nc
starting test for time6_lat200_lon360_Tair.nc
starting test for time6_lat1000_lon1800_Tair.nc
starting test for time6_lat100_lon180_Tair.nc
starting test for time6_lat2000_lon3600_Tair.nc
starting test for time6_lat200_lon360_Tair.nc
starting test for time6_lat1000_lon1800_Tair.nc
starting test for time6_lat100_lon180_Tair.nc
starting test for time6_lat2000_lon3600_Tair.nc
starting test for time6_lat200_lon360_Tair.nc


In [10]:
pd.DataFrame(open_and_load_times).sort_values(by='total_time')

filename                            chunk_shape  \
3     time6_lat200_lon360_Tair.nc    {'time': 6, 'lat': 200, 'lon': 360}   
1     time6_lat100_lon180_Tair.nc    {'time': 6, 'lat': 100, 'lon': 180}   
7     time6_lat200_lon360_Tair.nc    {'time': 6, 'lat': 200, 'lon': 360}   
5     time6_lat100_lon180_Tair.nc    {'time': 6, 'lat': 100, 'lon': 180}   
0   time6_lat1000_lon1800_Tair.nc  {'time': 6, 'lat': 1000, 'lon': 1800}   
11    time6_lat200_lon360_Tair.nc    {'time': 6, 'lat': 200, 'lon': 360}   
9     time6_lat100_lon180_Tair.nc    {'time': 6, 'lat': 100, 'lon': 180}   
4   time6_lat1000_lon1800_Tair.nc  {'time': 6, 'lat': 1000, 'lon': 1800}   
8   time6_lat1000_lon1800_Tair.nc  {'time': 6, 'lat': 1000, 'lon': 1800}   
12  time6_lat1000_lon1800_Tair.nc  {'time': 6, 'lat': 1000, 'lon': 1800}   
2   time6_lat2000_lon3600_Tair.nc  {'time': 6, 'lat': 2000, 'lon': 3600}   
6   time6_lat2000_lon3600_Tair.nc  {'time': 6, 'lat': 2000, 'lon': 3600}   
10  time6_lat2000_lon3600_Tair.nc  {'time': 6, 'lat': 2000, 'lon': 3600}   
15    time6_lat200_lon360_Tair.nc    {'time': 6, 'lat': 200, 'lon': 360}   
13    time6_lat100_lon180_Tair.nc    {'time': 6, 'lat': 100, 'lon': 180}   
14  time6_lat2000_lon3600_Tair.nc  {'time': 6, 'lat': 2000, 'lon': 3600}   
16  time6_lat1000_lon1800_Tair.nc  {'time': 6, 'lat': 1000, 'lon': 1800}   
18  time6_lat2000_lon3600_Tair.nc  {'time': 6, 'lat': 2000, 'lon': 3600}   
19    time6_lat200_lon360_Tair.nc    {'time': 6, 'lat': 200, 'lon': 360}   
17    time6_lat100_lon180_Tair.nc    {'time': 6, 'lat': 100, 'lon': 180}   

    chunk_size (mb)  open (seconds)       load  total_time  \
3          1.647949        0.076789   0.794996    0.871785   
1          0.411987        0.067649   0.975428    1.043077   
7          1.647949        0.107317   1.281932    1.389249   
5          0.411987        0.113110   1.423459    1.536569   
0         41.198730        0.182968   1.506731    1.689699   
11         1.647949        0.188760   1.575995    1.764755   
9          0.411987        0.184085   1.645278    1.829363   
4         41.198730        0.146871   1.707239    1.854110   
8         41.198730        0.211962   2.005777    2.217739   
12        41.198730        0.944120   4.668974    5.613094   
2        164.794922        0.088618   6.203344    6.291962   
6        164.794922        0.101338   6.219720    6.321058   
10       164.794922        0.167170   6.368314    6.535484   
15         1.647949        0.898357   6.048914    6.947271   
13         0.411987        0.897440   6.086757    6.984197   
14       164.794922        0.913705   8.580472    9.494177   
16        41.198730        1.866259   8.209763   10.076022   
18       164.794922        1.901452  11.507214   13.408666   
19         1.647949        1.759397  12.153174   13.912571   
17         0.411987        1.794954  12.466964   14.261918   

    s3fs_default_blocksize (mb)  
3                         0.954  
1                         0.954  
7                         4.768  
5                         4.768  
0                         0.954  
11                        9.537  
9                         9.537  
4                         4.768  
8                         9.537  
12                       47.684  
2                         0.954  
6                         4.768  
10                        9.537  
15                       47.684  
13                       47.684  
14                       47.684  
16                       95.367  
18                       95.367  
19                       95.367  
17                       95.367

# 5 degree x 5 degree timeseries

24 megabytes of data

In [11]:
# these 2 need to match, precision equaling the spatial size's decimal points
spatial_size = 5
precision = 0
s3_times = {}
open_and_load_times = []
lat_range, lon_range = test_helpers.select_random_spatial_range(spatial_size, precision)

In [12]:
for block_size in block_size_options:
    for test_file in test_files:
        s3fs_open_kwargs = {
            'default_fill_cache': False,
            'default_block_size': block_size,
        }
        s3fsfs = s3fs.S3FileSystem(**s3fs_open_kwargs)
        test_info, s3_request_info = test_helpers.load_series(test_file, s3fsfs, lat_range, lon_range)
        block_size_mb = round(block_size/1024/1024, 3)
        test_info['s3fs_default_blocksize (mb)'] = block_size_mb
        open_and_load_times.append(test_info)
        s3_times[str(block_size_mb)] = s3_request_info

starting test for time6_lat1000_lon1800_Tair.nc
starting test for time6_lat100_lon180_Tair.nc
starting test for time6_lat2000_lon3600_Tair.nc
starting test for time6_lat200_lon360_Tair.nc
starting test for time6_lat1000_lon1800_Tair.nc
starting test for time6_lat100_lon180_Tair.nc
starting test for time6_lat2000_lon3600_Tair.nc
starting test for time6_lat200_lon360_Tair.nc
starting test for time6_lat1000_lon1800_Tair.nc
starting test for time6_lat100_lon180_Tair.nc
starting test for time6_lat2000_lon3600_Tair.nc
starting test for time6_lat200_lon360_Tair.nc
starting test for time6_lat1000_lon1800_Tair.nc
starting test for time6_lat100_lon180_Tair.nc
starting test for time6_lat2000_lon3600_Tair.nc
starting test for time6_lat200_lon360_Tair.nc
starting test for time6_lat1000_lon1800_Tair.nc
starting test for time6_lat100_lon180_Tair.nc
starting test for time6_lat2000_lon3600_Tair.nc
starting test for time6_lat200_lon360_Tair.nc


In [13]:
pd.DataFrame(open_and_load_times).sort_values(by='total_time')

filename                            chunk_shape  \
3     time6_lat200_lon360_Tair.nc    {'time': 6, 'lat': 200, 'lon': 360}   
7     time6_lat200_lon360_Tair.nc    {'time': 6, 'lat': 200, 'lon': 360}   
1     time6_lat100_lon180_Tair.nc    {'time': 6, 'lat': 100, 'lon': 180}   
11    time6_lat200_lon360_Tair.nc    {'time': 6, 'lat': 200, 'lon': 360}   
4   time6_lat1000_lon1800_Tair.nc  {'time': 6, 'lat': 1000, 'lon': 1800}   
5     time6_lat100_lon180_Tair.nc    {'time': 6, 'lat': 100, 'lon': 180}   
0   time6_lat1000_lon1800_Tair.nc  {'time': 6, 'lat': 1000, 'lon': 1800}   
8   time6_lat1000_lon1800_Tair.nc  {'time': 6, 'lat': 1000, 'lon': 1800}   
9     time6_lat100_lon180_Tair.nc    {'time': 6, 'lat': 100, 'lon': 180}   
13    time6_lat100_lon180_Tair.nc    {'time': 6, 'lat': 100, 'lon': 180}   
6   time6_lat2000_lon3600_Tair.nc  {'time': 6, 'lat': 2000, 'lon': 3600}   
10  time6_lat2000_lon3600_Tair.nc  {'time': 6, 'lat': 2000, 'lon': 3600}   
2   time6_lat2000_lon3600_Tair.nc  {'time': 6, 'lat': 2000, 'lon': 3600}   
15    time6_lat200_lon360_Tair.nc    {'time': 6, 'lat': 200, 'lon': 360}   
12  time6_lat1000_lon1800_Tair.nc  {'time': 6, 'lat': 1000, 'lon': 1800}   
14  time6_lat2000_lon3600_Tair.nc  {'time': 6, 'lat': 2000, 'lon': 3600}   
18  time6_lat2000_lon3600_Tair.nc  {'time': 6, 'lat': 2000, 'lon': 3600}   
17    time6_lat100_lon180_Tair.nc    {'time': 6, 'lat': 100, 'lon': 180}   
19    time6_lat200_lon360_Tair.nc    {'time': 6, 'lat': 200, 'lon': 360}   
16  time6_lat1000_lon1800_Tair.nc  {'time': 6, 'lat': 1000, 'lon': 1800}   

    chunk_size (mb)  open (seconds)       load  total_time  \
3          1.647949        0.077130   2.006076    2.083206   
7          1.647949        0.108261   3.647080    3.755341   
1          0.411987        0.065596   3.758907    3.824503   
11         1.647949        0.176447   4.007733    4.184180   
4         41.198730        0.137084   4.395866    4.532950   
5          0.411987        0.114893   4.743132    4.858025   
0         41.198730        0.138139   4.723044    4.861183   
8         41.198730        0.217263   4.941018    5.158281   
9          0.411987        0.186808   5.688036    5.874844   
13         0.411987        0.915219   7.083240    7.998459   
6        164.794922        0.116230   7.951245    8.067475   
10       164.794922        0.175741   8.168233    8.343974   
2        164.794922        0.066127   9.964829   10.030956   
15         1.647949        0.886105   9.246283   10.132388   
12        41.198730        0.953009  10.206765   11.159774   
14       164.794922        0.901857  10.554055   11.455912   
18       164.794922        1.813209  12.813424   14.626633   
17         0.411987        1.876087  13.938915   15.815002   
19         1.647949        1.723682  14.378325   16.102007   
16        41.198730        1.778655  15.171746   16.950401   

    s3fs_default_blocksize (mb)  
3                         0.954  
7                         4.768  
1                         0.954  
11                        9.537  
4                         4.768  
5                         4.768  
0                         0.954  
8                         9.537  
9                         9.537  
13                       47.684  
6                         4.768  
10                        9.537  
2                         0.954  
15                       47.684  
12                       47.684  
14                       47.684  
18                       95.367  
17                       95.367  
19                       95.367  
16                       95.367

# 10 degree x 10 degree timeseries

96 megabytes of data

In [14]:
# these 2 need to match, precision equaling the spatial size's decimal points
spatial_size = 10
precision = 0
s3_times = {}
open_and_load_times = []
lat_range, lon_range = test_helpers.select_random_spatial_range(spatial_size, precision)

In [15]:
for block_size in block_size_options:
    for test_file in test_files:
        s3fs_open_kwargs = {
            'default_fill_cache': False,
            'default_block_size': block_size,
        }
        s3fsfs = s3fs.S3FileSystem(**s3fs_open_kwargs)
        test_info, s3_request_info = test_helpers.load_series(test_file, s3fsfs, lat_range, lon_range)
        block_size_mb = round(block_size/1024/1024, 3)
        test_info['s3fs_default_blocksize (mb)'] = block_size_mb
        open_and_load_times.append(test_info)
        s3_times[str(block_size_mb)] = s3_request_info

starting test for time6_lat1000_lon1800_Tair.nc
starting test for time6_lat100_lon180_Tair.nc
starting test for time6_lat2000_lon3600_Tair.nc
starting test for time6_lat200_lon360_Tair.nc
starting test for time6_lat1000_lon1800_Tair.nc
starting test for time6_lat100_lon180_Tair.nc
starting test for time6_lat2000_lon3600_Tair.nc
starting test for time6_lat200_lon360_Tair.nc
starting test for time6_lat1000_lon1800_Tair.nc
starting test for time6_lat100_lon180_Tair.nc
starting test for time6_lat2000_lon3600_Tair.nc
starting test for time6_lat200_lon360_Tair.nc
starting test for time6_lat1000_lon1800_Tair.nc
starting test for time6_lat100_lon180_Tair.nc
starting test for time6_lat2000_lon3600_Tair.nc
starting test for time6_lat200_lon360_Tair.nc
starting test for time6_lat1000_lon1800_Tair.nc
starting test for time6_lat100_lon180_Tair.nc
starting test for time6_lat2000_lon3600_Tair.nc
starting test for time6_lat200_lon360_Tair.nc


In [16]:
pd.DataFrame(open_and_load_times).sort_values(by='total_time')

filename                            chunk_shape  \
3     time6_lat200_lon360_Tair.nc    {'time': 6, 'lat': 200, 'lon': 360}   
7     time6_lat200_lon360_Tair.nc    {'time': 6, 'lat': 200, 'lon': 360}   
5     time6_lat100_lon180_Tair.nc    {'time': 6, 'lat': 100, 'lon': 180}   
11    time6_lat200_lon360_Tair.nc    {'time': 6, 'lat': 200, 'lon': 360}   
1     time6_lat100_lon180_Tair.nc    {'time': 6, 'lat': 100, 'lon': 180}   
2   time6_lat2000_lon3600_Tair.nc  {'time': 6, 'lat': 2000, 'lon': 3600}   
6   time6_lat2000_lon3600_Tair.nc  {'time': 6, 'lat': 2000, 'lon': 3600}   
0   time6_lat1000_lon1800_Tair.nc  {'time': 6, 'lat': 1000, 'lon': 1800}   
4   time6_lat1000_lon1800_Tair.nc  {'time': 6, 'lat': 1000, 'lon': 1800}   
10  time6_lat2000_lon3600_Tair.nc  {'time': 6, 'lat': 2000, 'lon': 3600}   
8   time6_lat1000_lon1800_Tair.nc  {'time': 6, 'lat': 1000, 'lon': 1800}   
9     time6_lat100_lon180_Tair.nc    {'time': 6, 'lat': 100, 'lon': 180}   
14  time6_lat2000_lon3600_Tair.nc  {'time': 6, 'lat': 2000, 'lon': 3600}   
12  time6_lat1000_lon1800_Tair.nc  {'time': 6, 'lat': 1000, 'lon': 1800}   
13    time6_lat100_lon180_Tair.nc    {'time': 6, 'lat': 100, 'lon': 180}   
15    time6_lat200_lon360_Tair.nc    {'time': 6, 'lat': 200, 'lon': 360}   
18  time6_lat2000_lon3600_Tair.nc  {'time': 6, 'lat': 2000, 'lon': 3600}   
19    time6_lat200_lon360_Tair.nc    {'time': 6, 'lat': 200, 'lon': 360}   
17    time6_lat100_lon180_Tair.nc    {'time': 6, 'lat': 100, 'lon': 180}   
16  time6_lat1000_lon1800_Tair.nc  {'time': 6, 'lat': 1000, 'lon': 1800}   

    chunk_size (mb)  open (seconds)       load  total_time  \
3          1.647949        0.082425   3.287946    3.370371   
7          1.647949        0.105990   4.106361    4.212351   
5          0.411987        0.108732   6.491827    6.600559   
11         1.647949        0.184591   6.876193    7.060784   
1          0.411987        0.068042   7.063942    7.131984   
2        164.794922        0.072374   7.717637    7.790011   
6        164.794922        0.102318   7.842083    7.944401   
0         41.198730        0.145399   7.991287    8.136686   
4         41.198730        0.142755   8.049571    8.192326   
10       164.794922        0.201007   8.031462    8.232469   
8         41.198730        0.212061   8.569646    8.781707   
9          0.411987        0.175819   9.750867    9.926686   
14       164.794922        0.883176  10.399552   11.282728   
12        41.198730        0.914596  11.821775   12.736371   
13         0.411987        0.897582  12.924874   13.822456   
15         1.647949        0.885257  13.689618   14.574875   
18       164.794922        1.769067  13.454118   15.223185   
19         1.647949        1.808493  14.920196   16.728689   
17         0.411987        1.768928  17.187350   18.956278   
16        41.198730        1.825459  18.653592   20.479051   

    s3fs_default_blocksize (mb)  
3                         0.954  
7                         4.768  
5                         4.768  
11                        9.537  
1                         0.954  
2                         0.954  
6                         4.768  
0                         0.954  
4                         4.768  
10                        9.537  
8                         9.537  
9                         9.537  
14                       47.684  
12                       47.684  
13                       47.684  
15                       47.684  
18                       95.367  
19                       95.367  
17                       95.367  
16                       95.367